## 1. 문서 임베딩(Embedding) 모델

- 임베딩(Embedding)은 텍스트, 이미지 등 다양한 데이터를 저차원의 밀집 벡터(dense vector)로 변환하는 기술
- 이 벡터 공간에서 의미적으로 유사한 항목들은 서로 가깝게 위치
- RAG 시스템에서는 질문(query)과 문서 청크(chunk)를 동일한 임베딩 모델을 사용하여 벡터로 변환한 뒤, 벡터 간 유사도(주로 코사인 유사도)를 계산하여 질문과 가장 관련성 높은 청크를 검색

**임베딩 모델 선택의 중요성:**
- **검색 품질**: 임베딩 모델의 성능은 검색 결과의 정확도에 직접적인 영향을 미치며, 문맥을 잘 이해하고 미묘한 의미 차이를 구분할 수 있는 모델이 좋음
- **다국어 지원**: 다국어 데이터를 처리해야 한다면 해당 언어들을 잘 지원하는 모델을 선택해야함
- **비용 및 속도**: API 기반 모델은 사용량에 따라 비용이 발생하고, 로컬 모델은 하드웨어 요구사항과 추론 속도를 고려해야함
- **컨텍스트 길이**: 임베딩 모델도 처리할 수 있는 최대 토큰 수가 정해져 있음(`embedding_ctx_length`). 분할된 청크가 이 길이를 넘지 않도록 주의해야 함

### 1-1 OpenAI 

- OpenAI는 고품질의 임베딩 모델을 API 형태로 제공함
- `text-embedding-3-small`, `text-embedding-3-large` 등이 있으며, 각각 성능, 비용, 차원 수 등에서 차이가 있음.

**장점:**
- **높은 성능**: 일반적으로 우수한 의미론적 표현 능력을 보여줌.
- **사용 편의성**: API 호출로 쉽게 사용할 수 있음.
- **최신 기술 반영**: 지속적으로 모델이 개선됨.

**단점:**
- **비용**: API 사용량에 따라 비용이 발생함.
- **데이터 프라이버시**: 데이터를 OpenAI 서버로 전송해야 하므로 민감한 정보 처리 시 주의가 필요
- **인터넷 연결 필수**: API 호출을 위해 인터넷 연결이 필요.

`(1) embedding 모델`

`Langchain_openai.OpenAIEmbeddings` 클래스를 사용하여 OpenAI 임베딩 모델을 초기화.
- `model`: 사용할 OpenAI 임베딩 모델의 이름을 지정합니다. (예: "text-embedding-3-small", "text-embedding-ada-002")

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
# 최신 모델 중 하나인 text-embedding-3-small 사용. 비용과 성능의 균형이 좋음.
embeddings_openai_small = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 객체 출력
print(embeddings_openai_small)

client=<openai.resources.embeddings.Embeddings object at 0x0000024A3AE699D0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000024A3AE7BC50> model='text-embedding-3-small' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [3]:
# 임베딩 모델의 컨텍스트 길이(최대 입력 토큰 수) 확인
# text-embedding-3-small, text-embedding-3-large, text-embedding-ada-002 등은 보통 8191 토큰입니다.
print(f"OpenAI 임베딩 모델 컨텍스트 길이: {embeddings_openai_small.embedding_ctx_length}")

OpenAI 임베딩 모델 컨텍스트 길이: 8191


`(2) embed_documents 사용`

- `embed_documents()` 메서드는 여러 개의 문서(텍스트 리스트)를 입력받아 각 문서에 대한 임베딩 벡터 리스트를 반환
- 벡터 저장소(Vector Store)에 문서를 저장하기 전에 이 메서드를 사용하여 임베딩을 생성

In [4]:
# 임베딩할 문서 컬렉션 (간단한 예시)
documents_for_embedding = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩 (API 호출 발생)
document_embeddings_openai = embeddings_openai_small.embed_documents(documents_for_embedding)

# 임베딩 결과 출력
print(f"임베딩된 벡터의 개수: {len(document_embeddings_openai)}")
print(f"첫 번째 벡터의 차원 (text-embedding-3-small은 1536차원): {len(document_embeddings_openai[0])}")
# print("첫 번째 문서의 임베딩 벡터 (일부만 출력):")
# print(document_embeddings_openai[0][:10]) # 너무 길어서 일부만

임베딩된 벡터의 개수: 5
첫 번째 벡터의 차원 (text-embedding-3-small은 1536차원): 1536


`(3) embed_query 사용`

- `embed_query()` 메서드는 단일 텍스트(주로 사용자 질문)를 입력받아 해당 텍스트에 대한 임베딩 벡터를 반환. 
- 이 벡터는 벡터 저장소에서 유사한 문서를 검색하는 데 사용

In [5]:
query_text_ko = "인공지능이란 무엇인가요?"
embedded_query_openai = embeddings_openai_small.embed_query(query_text_ko)

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
# print("쿼리 임베딩 벡터 (일부만 출력):")
# print(embedded_query_openai[:10])

쿼리 임베딩 벡터의 차원: 1536


`(4) 유사도 기반 검색`

- 두 벡터 간의 유사도는 주로 코사인 유사도(Cosine Similarity)로 측정 
- 코사인 유사도는 두 벡터가 이루는 각도의 코사인 값으로, -1에서 1 사이의 값을 가짐
- 1에 가까울수록 두 벡터(즉, 원본 텍스트)가 의미적으로 유사하다고 판단
- LangChain은 이러한 계산을 내부적으로 처리해주지만, 원리를 이해하는 것이 중요

`langchain_community.utils.math.cosine_similarity`는 입력으로 벡터들의 리스트를 받으므로 주의!!!! (예: `cosine_similarity([query_vector], [doc_vector1, doc_vector2, ...])` )

In [6]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수 (재사용 가능하도록 정의)
def find_most_similar(query_text: str, doc_embeddings: list, documents_text: list, embeddings_model_to_use):
    query_embedding = embeddings_model_to_use.embed_query(query_text)
    
    # 코사인 유사도 계산: query_embedding을 2D 배열로 만들어야 함
    # doc_embeddings도 이미 2D 배열 형태 (리스트의 리스트)로 가정
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0] # 결과는 2D 배열이므로 [0]으로 1D 배열 추출
    
    most_similar_idx = np.argmax(similarities) # 가장 높은 유사도 값의 인덱스
    return documents_text[most_similar_idx], similarities[most_similar_idx]

In [7]:
# 예제 쿼리
example_queries_ko = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]


In [8]:
# 각 쿼리에 대해 가장 유사한 문서 찾기 (OpenAI 임베딩 사용)
print("--- OpenAI 임베딩 기반 유사도 검색 결과 ---")
for q_text in example_queries_ko:
    most_similar_doc, similarity_score = find_most_similar(
        q_text, 
        document_embeddings_openai, # 위에서 생성한 OpenAI 문서 임베딩
        documents_for_embedding,    # 원본 문서 텍스트 리스트
        embeddings_openai_small     # 사용할 임베딩 모델
    )
    print(f"쿼리: {q_text}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity_score:.4f}")
    print()

--- OpenAI 임베딩 기반 유사도 검색 결과 ---
쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7117

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.6816

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.7051



### 1-2 Huggingface - 오픈소스 LLM

- Hugging Face Hub에는 수많은 오픈소스 임베딩 모델이 공개되어 있음. 
- `langchain_huggingface.embeddings.HuggingFaceEmbeddings` 클래스를 사용하면 이러한 모델들을 쉽게 LangChain과 통합할 수 있음. 
- 대표적으로 `sentence-transformers` 라이브러리 기반 모델들이 많이 사용되며, 한국어 특화 모델(예: `ko-sroberta-multitask`, `ko-sbert-nli`)이나 다국어 모델(예: `BAAI/bge-m3`, `paraphrase-multilingual-mpnet-base-v2`)을 선택할 수 있음.

**장점:**
- **무료 및 오픈소스**: 대부분의 모델을 무료로 사용할 수 있음.
- **다양한 선택지**: 특정 언어나 작업에 특화된 모델을 포함하여 선택의 폭이 넓음.
- **로컬 실행 가능**: 모델을 다운로드하여 로컬 환경에서 실행할 수 있어 데이터 프라이버시를 지키고 오프라인 사용이 가능.
- **모델 미세조정(Fine-tuning) 가능성**: 특정 데이터셋에 모델을 미세조정하여 성능을 더욱 향상시킬 수 있음 (일전에 논문을 읽은 기억으로는 추천을 하진 못하겠음, How Do Large Language Models Acquire Factual Knowledge During Pretraining?)

**단점:**
- **설정 및 관리**: 모델 다운로드, 의존성 관리 등 초기 설정이 필요
- **하드웨어 요구사항**: 고성능 모델은 GPU 등 충분한 컴퓨팅 자원을 요구할 가능성.
- **성능 편차**: 모델에 따라 성능 차이가 클 수 있으며, 최적의 모델을 찾기 위한 실험이 필요함.

`(1) embedding 모델`

- `model_name`: Hugging Face Hub에 등록된 모델의 이름을 지정 (예: "BAAI/bge-m3", "jhgan/ko-sroberta-multitask")
- `model_kwargs`: 모델 로드 시 전달할 추가 인자를 지정 `{'device': 'cuda'}`로 GPU 사용을 명시하거나, `{'device': 'cpu'}`로 CPU 사용을 명시할 수 있습니다. (기본적으로 사용 가능한 장치를 자동 감지하려 시도함)
- `encode_kwargs`: 임베딩 생성(인코딩) 시 전달할 추가 인자를 지정. `{'normalize_embeddings': True}`는 임베딩 벡터를 정규화하여 코사인 유사도 계산 시 성능을 높이는 데 도움이 될 수 있습니다 (모델에 따라 권장 여부 다름)

In [9]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face의 임베딩 모델 생성
# BAAI/bge-m3는 강력한 다국어 및 교차언어 임베딩 모델입니다.
embeddings_huggingface_bge_m3 = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3", 
    model_kwargs={'device': 'cpu'}, # CPU 사용 명시 (GPU 사용 가능하면 'cuda')
    encode_kwargs={'normalize_embeddings': True} # 임베딩 정규화 (bge 모델은 True 권장)
)

# 임베딩 객체 출력
print(embeddings_huggingface_bge_m3)

model_name='BAAI/bge-m3' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} multi_process=False show_progress=False


`(2) embed_documents 사용`

In [10]:
# 동일한 문서 컬렉션 사용 (documents_for_embedding)
# 문서 임베딩 (로컬에서 모델 추론 발생)
document_embeddings_huggingface = embeddings_huggingface_bge_m3.embed_documents(documents_for_embedding)

# 임베딩 결과 출력
print(f"임베딩된 벡터의 개수: {len(document_embeddings_huggingface)}")
print(f"첫 번째 벡터의 차원 (BAAI/bge-m3는 1024차원): {len(document_embeddings_huggingface[0])}")
# print("첫 번째 문서의 임베딩 벡터 (일부만 출력):")
# print(document_embeddings_huggingface[0][:10])

임베딩된 벡터의 개수: 5
첫 번째 벡터의 차원 (BAAI/bge-m3는 1024차원): 1024


`(3) embed_query 사용`

In [11]:
# 동일한 쿼리 텍스트 사용 (query_text_ko)
embedded_query_huggingface = embeddings_huggingface_bge_m3.embed_query(query_text_ko)

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_huggingface)}")
# print("쿼리 임베딩 벡터 (일부만 출력):")
# print(embedded_query_huggingface[:10])

쿼리 임베딩 벡터의 차원: 1024


`(4) 유사도 기반 검색`

앞서 정의한 `find_most_similar` 함수를 그대로 사용하여 Hugging Face 임베딩 모델 기반의 유사도 검색을 수행할 수 있음.

In [12]:
# 동일한 예제 쿼리 사용 (example_queries_ko)

# 각 쿼리에 대해 가장 유사한 문서 찾기 (HuggingFace BAAI/bge-m3 임베딩 사용)
print("--- HuggingFace (BAAI/bge-m3) 임베딩 기반 유사도 검색 결과 ---")
for q_text in example_queries_ko:
    most_similar_doc, similarity_score = find_most_similar(
        q_text, 
        document_embeddings_huggingface, # 위에서 생성한 HuggingFace 문서 임베딩
        documents_for_embedding,         # 원본 문서 텍스트 리스트
        embeddings_huggingface_bge_m3    # 사용할 임베딩 모델
    )
    print(f"쿼리: {q_text}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity_score:.4f}")
    print()

--- HuggingFace (BAAI/bge-m3) 임베딩 기반 유사도 검색 결과 ---
쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7269

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.7057

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.6843



### 1-3 Ollama - 오픈소스 LLM 로컬 실행 환경

- Ollama는 Llama 2, Mistral, Gemma 등 다양한 오픈소스 LLM 및 임베딩 모델을 로컬 환경에서 쉽게 실행하고 관리할 수 있게 해주는 도구
- `langchain_ollama.OllamaEmbeddings` 클래스를 통해 Ollama에서 실행 중인 임베딩 모델을 LangChain과 연동할 수 있음.

**선행 조건:**
- 로컬 머신에 Ollama가 설치되어 있고 실행 중이어야 함
- 사용할 임베딩 모델이 Ollama에 풀(pull)되어 있어야 함. (예: `ollama pull nomic-embed-text`, `ollama pull bge-m3`)

**장점:**
- **간편한 로컬 모델 관리**: 다양한 오픈소스 모델을 쉽게 다운로드하고 실행할 수 있음.
- **데이터 프라이버시**: 모든 처리가 로컬에서 이루어지므로 데이터 보안에 유리함.
- **오프라인 사용**: 인터넷 연결 없이도 사용 가능함

**단점:**
- **Ollama 설치 및 설정 필요**: Ollama를 먼저 설치하고 모델을 받아야 함.
- **하드웨어 의존성**: 로컬 머신의 성능(CPU, RAM, GPU)에 따라 속도와 안정성이 달라짐
- **모델 선택 제한**: Ollama에서 지원하는 모델로 제한될 수 있음.
- **한국어 성능**: 개인적으로 한국어 분석에 대해서 많이 부족한 경향 존재

`(1) embedding 모델`

- `model`: Ollama에 풀(pull)되어 있는 임베딩 모델의 이름을 지정 (예: "nomic-embed-text", "bge-m3")

In [13]:
from langchain_ollama import OllamaEmbeddings

# OllamaEmbeddings 모델 생성
# Ollama 서버에 'nomic-embed-text' 또는 'bge-m3' 모델이 pull 되어 있어야 합니다.
# 예: `ollama pull nomic-embed-text` 또는 `ollama pull bge-m3` 실행 후 사용
try:
    # embeddings_ollama_nomic = OllamaEmbeddings(model="nomic-embed-text")
    embeddings_ollama_bge_m3 = OllamaEmbeddings(model="bge-m3") # bge-m3가 더 강력할 수 있음
    print(embeddings_ollama_bge_m3)
    ollama_ready = True
except Exception as e:
    print(f"Ollama 연결 또는 모델 로드 실패: {e}")
    print("Ollama 예제를 실행하려면 Ollama 서버를 실행하고 'bge-m3' 모델을 pull 해주세요. (예: ollama run bge-m3)")
    ollama_ready = False

model='bge-m3' base_url=None client_kwargs={} async_client_kwargs={} sync_client_kwargs={} mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None keep_alive=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None


`(2) embed_documents 사용`

In [14]:
document_embeddings_ollama = [] # 초기화
if ollama_ready:
    # 동일한 문서 컬렉션 사용 (documents_for_embedding)
    # 문서 임베딩 (로컬 Ollama 서버와 통신하여 추론 발생)
    document_embeddings_ollama = embeddings_ollama_bge_m3.embed_documents(documents_for_embedding)

    # 임베딩 결과 출력
    print(f"임베딩된 벡터의 개수: {len(document_embeddings_ollama)}")
    if document_embeddings_ollama:
        print(f"첫 번째 벡터의 차원 (Ollama bge-m3는 1024차원): {len(document_embeddings_ollama[0])}")
        # print("첫 번째 문서의 임베딩 벡터 (일부만 출력):")
        # print(document_embeddings_ollama[0][:10])
else:
    print("Ollama가 준비되지 않아 문서 임베딩을 건너뜁니다.")

임베딩된 벡터의 개수: 5
첫 번째 벡터의 차원 (Ollama bge-m3는 1024차원): 1024


`(3) embed_query 사용`

In [15]:
embedded_query_ollama = None # 초기화
if ollama_ready:
    # 동일한 쿼리 텍스트 사용 (query_text_ko)
    embedded_query_ollama = embeddings_ollama_bge_m3.embed_query(query_text_ko)

    # 쿼리 임베딩 결과 출력
    if embedded_query_ollama:
        print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_ollama)}")
        # print("쿼리 임베딩 벡터 (일부만 출력):")
        # print(embedded_query_ollama[:10])
else:
    print("Ollama가 준비되지 않아 쿼리 임베딩을 건너뜁니다.")

쿼리 임베딩 벡터의 차원: 1024


`(4) 유사도 기반 검색`

In [16]:
if ollama_ready and document_embeddings_ollama: # 문서 임베딩이 성공적으로 수행되었는지 확인
    # 동일한 예제 쿼리 사용 (example_queries_ko)

    # 각 쿼리에 대해 가장 유사한 문서 찾기 (Ollama bge-m3 임베딩 사용)
    print("--- Ollama (bge-m3) 임베딩 기반 유사도 검색 결과 ---")
    for q_text in example_queries_ko:
        most_similar_doc, similarity_score = find_most_similar(
            q_text, 
            document_embeddings_ollama,    # 위에서 생성한 Ollama 문서 임베딩
            documents_for_embedding,       # 원본 문서 텍스트 리스트
            embeddings_ollama_bge_m3       # 사용할 임베딩 모델
        )
        print(f"쿼리: {q_text}")
        print(f"가장 유사한 문서: {most_similar_doc}")
        print(f"유사도: {similarity_score:.4f}")
        print()
else:
    print("Ollama가 준비되지 않았거나 문서 임베딩에 실패하여 유사도 검색을 건너뜁니다.")

--- Ollama (bge-m3) 임베딩 기반 유사도 검색 결과 ---
쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7270

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.7048

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.6836

